In [1]:
import folium

print(folium.__file__)
print(folium.__version__)

/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/folium/__init__.py
0.2.1


### Make a figure

In [2]:
import numpy as np
import pandas as pd
import numpy.ma as ma


def make_data():
    x = np.linspace(-np.pi, np.pi, 101)
    sin = np.sin(x)
    cos = np.cos(x)
    cos[20:50] = np.NaN
    return pd.DataFrame(np.asanyarray([sin, cos]).T, columns=['sin', 'cos'], index=x)

df = make_data()
resolution, width, height = 75, 7, 3

### Instantiate the map

In [3]:
station = '42'
lon, lat = -42, -21
mapa = folium.Map(location=[lat, lon], zoom_start=5)

### PNG

In [4]:
import base64
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(width, height))
ax = df.plot(ax=ax, legend=False)
ax.set_ylabel('Sea surface height (m)')
png = 'mpld3_{}.png'.format(station)
fig.savefig(png, dpi=resolution)

encoded = base64.b64encode(open(png, 'rb').read())

In [5]:
from folium.element import IFrame

html = '<img src="data:image/png;base64,{}">'.format
iframe = IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
popup = folium.Popup(iframe, max_width=2650)

icon = folium.Icon(color="red", icon="ok")
marker = folium.Marker(location=[lat-2, lon-1], popup=popup, icon=icon)
mapa.add_children(marker);

### Vega/Vincent

In [6]:
import json
import vincent

df.fillna(value='null', inplace=True)  # Does not handle missing values.
vis = vincent.Line(df, width=width*resolution, height=height*resolution)
vis.legend(title='Vega');

In [7]:
vega = folium.Vega(json.loads(vis.to_json()), width="100%", height="100%")
popup = folium.Popup(max_width=vis.width+75).add_child(vega)

icon = folium.Icon(color="green", icon="ok")
marker = folium.Marker(location=[lat-1, lon+1], popup=popup, icon=icon)

mapa.add_children(marker);

### mpld3

In [8]:
import mpld3
from folium.element import IFrame
from mpld3.plugins import LineLabelTooltip, connect


fig, ax = plt.subplots(figsize=(width, height))
ax = df.plot(ax=ax, legend=False)
ax.set_ylabel('Sea surface height (m)')

[mpld3.plugins.connect(fig, LineLabelTooltip(line, name)) for
 line, name in zip(ax.lines, df.columns)]

figname = 'mpld3_{}.html'.format(station)
mpld3.save_html(fig, figname)

In [9]:
with open(figname, 'r') as f:
    html = f.read()

iframe = IFrame(html, width=(width*resolution)+75, height=(height*resolution)+50)
popup = folium.Popup(iframe, max_width=2650)

icon = folium.Icon(color="blue", icon="ok")
marker = folium.Marker(location=[lat+1, lon-1], popup=popup, icon=icon)

mapa.add_children(marker);

In [10]:
mapa

/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/home/filipe/.virtualenvs/

TODO: Bokeh